# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Experiment, Workspace

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneProject'

experiment=Experiment(ws, experiment_name)

## Creating Compute Cluster

In [3]:
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "computeluster"
try:
    compute_cluster = ComputeTarget(ws, cluster_name)
    print("Found exisitng compute cluster")
except ComputeTargetException:
    print("Creating new computer cluster")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)

    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    print("Created computer cluster")

Found exisitng compute cluster


## Importing Dataset

In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory

ds = TabularDatasetFactory.from_delimited_files(path='https://raw.githubusercontent.com/gaganmanku96/Capstone-Project-Azure-Machine-Learning/master/heart_failure_clinical_records_dataset.csv')
df = ds.to_pandas_dataframe()
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.00,0,582,0,20,1,265000.00,1.90,130,1,0,4,1
1,55.00,0,7861,0,38,0,263358.03,1.10,136,1,0,6,1
2,65.00,0,146,0,20,0,162000.00,1.30,129,1,1,7,1
3,50.00,1,111,0,20,0,210000.00,1.90,137,1,0,7,1
4,65.00,1,160,1,20,0,327000.00,2.70,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [15]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "experiment_timeout_hours": 0.3,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "verbosity": logging.INFO
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    training_data=df,
    label_column_name='DEATH_EVENT',
    **automl_settings
    )

In [16]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [19]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [20]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: CapstoneProject,
Id: AutoML_71e10d2f-fa78-46af-b279-7f3c4f9555f6_33,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                             KNeighborsClassifier(algorithm='auto',
                                                                                                  leaf_size=30,
                                                

In [21]:
#TODO: Save the best model
best_run.download_file('/outputs/model.pkl', 'automl_best_model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [4]:
from azureml.core.model import Model

model = Model.register(ws, model_path='./automl_best_model.pkl', model_name='heart_disease')

Registering model heart_disease


In [20]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import LocalWebservice, AciWebservice

myenv = Environment.get(ws, name='AzureML-AutoML')
# myenv = Environment.from_pip_requirements(name = "myenv",
#                                           file_path = "requirements.txt")     

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)
# deployment_config = LocalWebservice.deploy_configuration(port=8000)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True, auth_enabled=True)


service = Model.deploy(ws, name='heartservicehealth', models=[model],inference_config=inference_config, deployment_config=aci_config, overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [33]:
primary_key, secondry_key = service.get_keys()

In [44]:
import requests
import json

uri = service.scoring_uri
data = {"data": [{"age":60.000000,"anaemia":0.000000,"creatinine_phosphokinase":250.000000,"diabetes":0.000000,"ejection_fraction":38.000000,"high_blood_pressure":0.000000,"platelets":262000.000000,"serum_creatinine":1.10000,"serum_sodium":137.000000,"sex":1.000000,"smoking":0.00000,"time":115.000000}]}

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {primary_key}'

payload = json.dumps(data)
respone = requests.post(uri, payload, headers=headers)
print(respone.text)

{}
{}


TODO: In the cell below, print the logs of the web service and delete the service

In [46]:
dep_logs = service.get_logs()
for l in dep_logs.split('\n'):
    print(l)

2021-01-31T21:16:42,120650614+00:00 - iot-server/run 
2021-01-31T21:16:42,121260159+00:00 - gunicorn/run 
2021-01-31T21:16:42,125232651+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7ade26eb614f97df8030bc480da59236/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-01-31T21:16:42,128191368+00:00 - rsyslog/run 
rsyslogd

In [47]:
service.delete()